In [1]:
%load_ext tensorboard

In [2]:
!ls | grep emgdat.zip || curl https://archive.ics.uci.edu/ml/machine-learning-databases/00481/EMG_data_for_gestures-master.zip --output emgdat.zip 
!rm -r ./logs
   

"""
    If you don't have linux or 7z 
    then just download dataset from repo url and extract manually 
    
    or 
    
    install 7z and or nice linux distro (I use arch :) 
"""

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 16.8M  100 16.8M    0     0  2473k      0  0:00:06  0:00:06 --:--:-- 2838k
rm: cannot remove './logs': No such file or directory


"\n    If you don't have linux or 7z \n    then just download dataset from repo url and extract manually \n    \n    or \n    \n    install 7z and or nice linux distro (I use arch :) \n"

In [3]:
!ls | grep EMG_data_for_gestures-master || 7z x emgdat.zip 


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,12 CPUs Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz (906EA),ASM,AES-NI)

Scanning the drive for archives:
  0M Sca        1 file, 17699840 bytes (17 MiB)

Extracting archive: emgdat.zip
--
Path = emgdat.zip
Type = zip
Physical Size = 17699840
Comment = 43e8a69d7465e9bd4d22a71ed9cf621cf8d1d7b2

     33% 36 - EMG_data_for_gestures-master/12/2_raw_data_11-36_28.03.16.tx                                                                       68% 72 - EMG_data_for_gestures-master/24/2_raw_data_10-17_12.04.16.tx                                                                      Everything is Ok

Folders: 37
Files: 73
Size:       263003311
Compressed: 17699840


In [4]:
!cd ./EMG_data_for_gestures-master/ && rm -r README.txt 36 35 34 33 
#Some of the data seems to be corrupted so delete the last few 

In [5]:
from pal import feature_normalize, gen_numpy_datlab

gen_numpy_datlab('./EMG_data_for_gestures-master/') # Generate binary caches of dataset. See pal.py for details

In [6]:
from emghyp import HyperEmg
import pywt
import numpy as np
from tensorflow.keras.layers import Conv1D, Dropout, MaxPooling1D, GlobalAveragePooling1D, Dense  
import tensorflow as tf
import tensorflow.keras as ks
from tensorboard.plugins.hparams import api as hp

In [7]:
"""
These are two preprocessing functions to pass to the EmgHyp class

both use pywavelets to decompose the signals into wavelet trasforms and then normalize for use in convnet. 
get_wavelets1D uses the 1D discrete wavelet transform and 
get_wavelets2D uses the 2d dwt 
"""

def get_wavelets1D(data):
    wavelets = []
    for sample in data:
        pywav = []
        (a, b) = pywt.dwt(sample, 'db2')
        
        pywav.extend(a)
        pywav.extend(b)
                    
        wavelets.append(pywav)
    
    return feature_normalize(np.array(wavelets))

def get_wavelets2D(data):
    wavelets = []
    for sample in data:
        pywav = []
        (a, (b, c, d)) = pywt.dwt2(sample, 'db2')
        
        pywav.extend(a)
        pywav.extend(b)
        pywav.extend(c)
        pywav.extend(d)
                    
        wavelets.append(pywav)
        
    return feature_normalize(np.array(wavelets))

prepropfs = [get_wavelets1D, get_wavelets2D] # Preprocessing hyperparameters to pass to HyperEmg

In [8]:
"""
This is a list of hyperparameters you use. 
You can uncomment to have more models tested, but the time complexity will go up exponentially since
we are using a grid search. 

This much is already a lot for my lab(ptop), an RTX 2070 mobile non max-q
"""
hps = [
        hp.HParam('filter1', hp.Discrete(list(range(50, 400, 100)))),
        hp.HParam('kernel1', hp.Discrete(list(range(10, 60, 20)))),

        #hp.HParam('filter2', hp.Discrete(list(range(50, 400)))),
        #hp.HParam('kernel2', hp.Discrete(list(range(5, 30)))),

        #hp.HParam('filter3', hp.Discrete(list(range(50, 400)))),
        #hp.HParam('kernel3', hp.Discrete(list(range(5, 30)))),

        hp.HParam('dense1', hp.Discrete(list(range(50, 400, 100)))),
        #hp.HParam('dense2', hp.Discrete(list(range(50, 400, 100)))),
    
        hp.HParam('preprocessing', hp.Discrete([get_wavelets1D.__name__, get_wavelets2D.__name__])),

        #hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
       ]



In [9]:
"""
Create an anonymous function that returns a model.
The model will use the hyperparameter variables you defined above as parameters 
rather than constant values.
"""
mf = lambda hparams, ishp : ks.models.Sequential(
        [
        Conv1D(hparams['filter1'],
               hparams['kernel1'],
               strides = 2, padding = 'same', activation = 'relu', input_shape=(ishp)
              ),
        Dropout(.3),
        MaxPooling1D(3),

        Conv1D(hparams['filter1'],
               hparams['kernel1'] // 2,
               strides = 2, padding = 'same', activation = 'relu'
              ),
        Dropout(.3),
        MaxPooling1D(3),

        Conv1D(hparams['filter1'],
               hparams['kernel1'] // 4,
               strides = 2, padding = 'same', activation = 'relu'
              ),
        Dropout(.3),
        MaxPooling1D(3),
      
        GlobalAveragePooling1D(),

        Dense(hparams['dense1'], activation = 'relu'),
        Dropout(.3),


        Dense(hparams['dense1'], activation = 'relu'),
        Dropout(.3),

        Dense(7, activation = 'softmax')
    ])

In [10]:
HyperEmg(hps, mf, prepropfs)

dict_keys(['get_wavelets1D', 'get_wavelets2D'])
--- Starting trial: run-1
Epoch 1/45
150/150 [==============================] - 2s 16ms/step - loss: 1.5902 - accuracy: 0.3764 - val_loss: 1.2730 - val_accuracy: 0.5755
Epoch 2/45
150/150 [==============================] - 0s 3ms/step - loss: 1.0492 - accuracy: 0.5758 - val_loss: 0.9492 - val_accuracy: 0.6547
Epoch 3/45
150/150 [==============================] - 0s 3ms/step - loss: 0.9545 - accuracy: 0.6199 - val_loss: 0.8774 - val_accuracy: 0.6831
Epoch 4/45
150/150 [==============================] - 0s 3ms/step - loss: 0.8867 - accuracy: 0.6499 - val_loss: 0.8136 - val_accuracy: 0.6964
Epoch 5/45
150/150 [==============================] - 0s 3ms/step - loss: 0.8469 - accuracy: 0.6647 - val_loss: 0.8843 - val_accuracy: 0.6105
Epoch 6/45
150/150 [==============================] - 0s 3ms/step - loss: 0.8022 - accuracy: 0.6948 - val_loss: 0.7878 - val_accuracy: 0.7456
Epoch 7/45
150/150 [==============================] - 1s 5ms/step - loss:

Epoch 13/45
150/150 [==============================] - 1s 4ms/step - loss: 0.5721 - accuracy: 0.7953 - val_loss: 0.5729 - val_accuracy: 0.8148
Epoch 14/45
150/150 [==============================] - 1s 4ms/step - loss: 0.5541 - accuracy: 0.7993 - val_loss: 0.5986 - val_accuracy: 0.7982
Epoch 15/45
150/150 [==============================] - 1s 4ms/step - loss: 0.5215 - accuracy: 0.8126 - val_loss: 0.6052 - val_accuracy: 0.8007
Epoch 16/45
150/150 [==============================] - 1s 3ms/step - loss: 0.5122 - accuracy: 0.8199 - val_loss: 0.6267 - val_accuracy: 0.8007
Epoch 17/45
150/150 [==============================] - 1s 4ms/step - loss: 0.4987 - accuracy: 0.8206 - val_loss: 0.5496 - val_accuracy: 0.8332
Epoch 18/45
150/150 [==============================] - 1s 4ms/step - loss: 0.4607 - accuracy: 0.8323 - val_loss: 0.6361 - val_accuracy: 0.7673
Epoch 19/45
150/150 [==============================] - 1s 4ms/step - loss: 0.4712 - accuracy: 0.8327 - val_loss: 0.5486 - val_accuracy: 0.8148

150/150 [==============================] - 1s 6ms/step - loss: 0.2756 - accuracy: 0.8948 - val_loss: 0.6421 - val_accuracy: 0.7781
Epoch 38/45
150/150 [==============================] - 1s 6ms/step - loss: 0.2377 - accuracy: 0.9170 - val_loss: 0.6043 - val_accuracy: 0.7998
Epoch 39/45
150/150 [==============================] - 1s 6ms/step - loss: 0.2456 - accuracy: 0.9115 - val_loss: 0.6040 - val_accuracy: 0.8048
Epoch 40/45
150/150 [==============================] - 1s 6ms/step - loss: 0.2497 - accuracy: 0.9126 - val_loss: 0.5734 - val_accuracy: 0.7957
Epoch 41/45
150/150 [==============================] - 1s 6ms/step - loss: 0.2413 - accuracy: 0.9151 - val_loss: 0.6188 - val_accuracy: 0.8065
Epoch 42/45
150/150 [==============================] - 1s 7ms/step - loss: 0.2588 - accuracy: 0.9086 - val_loss: 0.5886 - val_accuracy: 0.8015
Epoch 43/45
150/150 [==============================] - 1s 7ms/step - loss: 0.1969 - accuracy: 0.9332 - val_loss: 0.5566 - val_accuracy: 0.8123
Epoch 44/45

150/150 [==============================] - 1s 8ms/step - loss: 0.4878 - accuracy: 0.8262 - val_loss: 0.6105 - val_accuracy: 0.7765
Epoch 17/45
150/150 [==============================] - 1s 8ms/step - loss: 0.4666 - accuracy: 0.8314 - val_loss: 0.5519 - val_accuracy: 0.8182
Epoch 18/45
150/150 [==============================] - 1s 7ms/step - loss: 0.4788 - accuracy: 0.8272 - val_loss: 0.5958 - val_accuracy: 0.8057
Epoch 19/45
150/150 [==============================] - 1s 8ms/step - loss: 0.4722 - accuracy: 0.8331 - val_loss: 0.6121 - val_accuracy: 0.7923
Epoch 20/45
150/150 [==============================] - 1s 8ms/step - loss: 0.4655 - accuracy: 0.8346 - val_loss: 0.5849 - val_accuracy: 0.7948
Epoch 21/45
150/150 [==============================] - 2s 10ms/step - loss: 0.4557 - accuracy: 0.8358 - val_loss: 0.5753 - val_accuracy: 0.7865
Epoch 22/45
150/150 [==============================] - 1s 8ms/step - loss: 0.4307 - accuracy: 0.8435 - val_loss: 0.6155 - val_accuracy: 0.8073
Epoch 23/4

150/150 [==============================] - 2s 10ms/step - loss: 0.4107 - accuracy: 0.8517 - val_loss: 0.6325 - val_accuracy: 0.7590
Epoch 29/45
150/150 [==============================] - 2s 12ms/step - loss: 0.4641 - accuracy: 0.8371 - val_loss: 0.6101 - val_accuracy: 0.7857
Epoch 30/45
150/150 [==============================] - 2s 11ms/step - loss: 0.4423 - accuracy: 0.8471 - val_loss: 0.5997 - val_accuracy: 0.7865
Epoch 31/45
150/150 [==============================] - 2s 11ms/step - loss: 0.3959 - accuracy: 0.8596 - val_loss: 0.6155 - val_accuracy: 0.7723
Epoch 32/45
150/150 [==============================] - 2s 14ms/step - loss: 0.3892 - accuracy: 0.8565 - val_loss: 0.6820 - val_accuracy: 0.7548
Epoch 33/45
150/150 [==============================] - 2s 10ms/step - loss: 0.3480 - accuracy: 0.8681 - val_loss: 0.6234 - val_accuracy: 0.8040
Epoch 34/45
150/150 [==============================] - 2s 12ms/step - loss: 0.4008 - accuracy: 0.8558 - val_loss: 0.6327 - val_accuracy: 0.7773
Epoc

...

150/150 [==============================] - 1s 5ms/step - loss: 0.3523 - accuracy: 0.8665 - val_loss: 0.5672 - val_accuracy: 0.7973
Epoch 29/45
150/150 [==============================] - 1s 4ms/step - loss: 0.3347 - accuracy: 0.8769 - val_loss: 0.5354 - val_accuracy: 0.7898
Epoch 30/45
150/150 [==============================] - 1s 5ms/step - loss: 0.3180 - accuracy: 0.8767 - val_loss: 0.5759 - val_accuracy: 0.7998
Epoch 31/45
150/150 [==============================] - 1s 5ms/step - loss: 0.3305 - accuracy: 0.8725 - val_loss: 0.5678 - val_accuracy: 0.7948
Epoch 32/45
150/150 [==============================] - 1s 6ms/step - loss: 0.3158 - accuracy: 0.8842 - val_loss: 0.6549 - val_accuracy: 0.7590
Epoch 33/45
150/150 [==============================] - 1s 5ms/step - loss: 0.3059 - accuracy: 0.8792 - val_loss: 0.5526 - val_accuracy: 0.8115
Epoch 34/45
150/150 [==============================] - 1s 5ms/step - loss: 0.3086 - accuracy: 0.8832 - val_loss: 0.5629 - val_accuracy: 0.7973
Epoch 35/45

150/150 [==============================] - 1s 8ms/step - loss: 0.5120 - accuracy: 0.8151 - val_loss: 0.5534 - val_accuracy: 0.8232
Epoch 8/45
150/150 [==============================] - 1s 8ms/step - loss: 0.4945 - accuracy: 0.8141 - val_loss: 0.6092 - val_accuracy: 0.8157
Epoch 9/45
150/150 [==============================] - 1s 7ms/step - loss: 0.4607 - accuracy: 0.8258 - val_loss: 0.5517 - val_accuracy: 0.8007
Epoch 10/45
150/150 [==============================] - 1s 7ms/step - loss: 0.4140 - accuracy: 0.8427 - val_loss: 0.6015 - val_accuracy: 0.7848
Epoch 11/45
150/150 [==============================] - 1s 8ms/step - loss: 0.3949 - accuracy: 0.8510 - val_loss: 0.6439 - val_accuracy: 0.7523
Epoch 12/45
150/150 [==============================] - 1s 8ms/step - loss: 0.3716 - accuracy: 0.8604 - val_loss: 0.5372 - val_accuracy: 0.8190
Epoch 13/45
150/150 [==============================] - 1s 6ms/step - loss: 0.3396 - accuracy: 0.8694 - val_loss: 0.5858 - val_accuracy: 0.7815
Epoch 14/45
1

150/150 [==============================] - 1s 6ms/step - loss: 0.2568 - accuracy: 0.8999 - val_loss: 0.6576 - val_accuracy: 0.7807
Epoch 32/45
150/150 [==============================] - 1s 6ms/step - loss: 0.2366 - accuracy: 0.9145 - val_loss: 0.7232 - val_accuracy: 0.7490
Epoch 33/45
150/150 [==============================] - 1s 8ms/step - loss: 0.2752 - accuracy: 0.8980 - val_loss: 0.6519 - val_accuracy: 0.7882
Epoch 34/45
150/150 [==============================] - 1s 5ms/step - loss: 0.2485 - accuracy: 0.9069 - val_loss: 0.6290 - val_accuracy: 0.7823
Epoch 35/45
150/150 [==============================] - 1s 6ms/step - loss: 0.2548 - accuracy: 0.9034 - val_loss: 0.6012 - val_accuracy: 0.7990
Epoch 36/45
150/150 [==============================] - 1s 6ms/step - loss: 0.2251 - accuracy: 0.9157 - val_loss: 0.7020 - val_accuracy: 0.7731
Epoch 37/45
150/150 [==============================] - 1s 7ms/step - loss: 0.2199 - accuracy: 0.9174 - val_loss: 0.7325 - val_accuracy: 0.7765
Epoch 38/45

Epoch 10/45
150/150 [==============================] - 1s 6ms/step - loss: 0.4463 - accuracy: 0.8293 - val_loss: 0.6374 - val_accuracy: 0.7498
Epoch 11/45
150/150 [==============================] - 1s 5ms/step - loss: 0.4162 - accuracy: 0.8433 - val_loss: 0.5477 - val_accuracy: 0.7882
Epoch 12/45
150/150 [==============================] - 1s 6ms/step - loss: 0.3989 - accuracy: 0.8508 - val_loss: 0.6509 - val_accuracy: 0.7515
Epoch 13/45
150/150 [==============================] - 1s 6ms/step - loss: 0.4075 - accuracy: 0.8448 - val_loss: 0.6599 - val_accuracy: 0.7690
Epoch 14/45
150/150 [==============================] - 1s 7ms/step - loss: 0.3830 - accuracy: 0.8527 - val_loss: 0.5559 - val_accuracy: 0.7882
Epoch 15/45
150/150 [==============================] - 1s 5ms/step - loss: 0.3596 - accuracy: 0.8638 - val_loss: 0.6911 - val_accuracy: 0.7556
Epoch 16/45
150/150 [==============================] - 1s 7ms/step - loss: 0.3762 - accuracy: 0.8585 - val_loss: 0.7544 - val_accuracy: 0.7231

150/150 [==============================] - 1s 9ms/step - loss: 0.2081 - accuracy: 0.9253 - val_loss: 0.8289 - val_accuracy: 0.7473
Epoch 35/45
150/150 [==============================] - 1s 9ms/step - loss: 0.2151 - accuracy: 0.9259 - val_loss: 0.9347 - val_accuracy: 0.7590
Epoch 36/45
150/150 [==============================] - 1s 9ms/step - loss: 0.2026 - accuracy: 0.9249 - val_loss: 1.0571 - val_accuracy: 0.7456
Epoch 37/45
150/150 [==============================] - 1s 9ms/step - loss: 0.2365 - accuracy: 0.9197 - val_loss: 0.7263 - val_accuracy: 0.7798
Epoch 38/45
150/150 [==============================] - 1s 10ms/step - loss: 0.2032 - accuracy: 0.9257 - val_loss: 0.6611 - val_accuracy: 0.7907
Epoch 39/45
150/150 [==============================] - 2s 10ms/step - loss: 0.2323 - accuracy: 0.9195 - val_loss: 0.9234 - val_accuracy: 0.7615
Epoch 40/45
150/150 [==============================] - 1s 9ms/step - loss: 0.2290 - accuracy: 0.9199 - val_loss: 0.8176 - val_accuracy: 0.7590
Epoch 41/

In [11]:
%tensorboard --logdir logs/hparam_tuning